# A Table of Temperatures

In [1]:
import pandas as pd
import psutil
with __import__("importnb").Notebook():
    from ScrudTableHandler import ScrudTableHandler

> At present, we return just one handler, but could eventually see the routes organized in a different way.

In [2]:
def sense():
    temps = psutil.sensors_temperatures()
    all_temps = [
        {
            "category": _cat, 
            "label": temp.label, 
            "current": temp.current, 
            "high": temp.high, 
            "critical": temp.critical
        }
        for _cat, cat_temps in temps.items()
        for temp in cat_temps
    ]
    df = pd.DataFrame(all_temps)
    return {
        "df": df,
        "context": {},
        "schema": {},
        "frame": {}
    }

In [3]:
class TemperatureTableHandler(ScrudTableHandler):
    async def value(self):
        self._parts = sense()
        return await super().value()

In [4]:
def default_handlers(full_url, base_url, parts=None, prefix="some-dataframe"):
    parts = parts or sense()
    return [(
        base_url + prefix + r'/?(?P<doc>@context|\$schema)?/?', 
        TemperatureTableHandler, 
        dict(parts=parts, url_base=f"{full_url}{prefix}/")
    )]